In [1]:
import os
import gc
import json
import warnings
import asyncio
import nest_asyncio
import subprocess
import torch
import customtkinter as ctk
from tkinter import filedialog

# Library สำหรับ AI
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import edge_tts
from moviepy.editor import AudioFileClip, concatenate_audioclips

c:\Users\aungl\anaconda3\envs\ai_work\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ---------------------------------------------------------
# SETUP & CONFIG
# ---------------------------------------------------------
warnings.filterwarnings("ignore")
try:
    nest_asyncio.apply()
except:
    pass

# ตรวจสอบ Device
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"⚙️ Running on: {DEVICE}")

⚙️ Running on: cuda:0


In [3]:
# ---------------------------------------------------------
# HELPER FUNCTIONS (เครื่องมือช่วย)
# ---------------------------------------------------------
def cleanup_gpu():
    """ฟังก์ชันล้างขยะใน GPU เพื่อป้องกัน VRAM เต็ม"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

def get_duration_ffmpeg(file_path):
    """ดึงความยาวไฟล์แบบปลอดภัย"""
    try:
        if not os.path.exists(file_path):
            return None
        
        cmd = [
            "ffprobe", "-v", "error", "-show_entries", "format=duration", 
            "-of", "json", file_path
        ]
        # creationflags=0x08000000 ใช้ซ่อนหน้าต่าง console บน Windows
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        data = json.loads(result.stdout)
        return float(data['format']['duration'])
    except Exception as e:
        print(f"⚠️ Warning reading duration: {e}")
        return None

In [4]:
# ---------------------------------------------------------
# 1. SPEECH TO TEXT (Whisper)
# ---------------------------------------------------------
def speech_to_text_en(audio_path):
    print("------------------------------------------------")
    print("1️⃣ Step 1: Speech to Text (Whisper Large-V3)")
    
    temp_wav = "temp_extracted_audio.wav"
    
    try:
        # 1.1 Extract Audio using FFmpeg (เบากว่า MoviePy)
        if audio_path.lower().endswith(('.mp4', '.mov', '.avi', '.mkv', '.webm')):
            print(f"   🎬 Extracting audio from video...")
            subprocess.run([
                "ffmpeg", "-y", "-i", audio_path, 
                "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", 
                temp_wav
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
            target_file = temp_wav
        else:
            target_file = audio_path

        # 1.2 Load Model (Load ตรงนี้เพื่อประหยัด RAM ก่อนหน้านี้)
        print(f"   🎙️ Loading Whisper Model...")
        dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        pipe = pipeline(
            "automatic-speech-recognition",
            model="openai/whisper-large-v3",
            torch_dtype=dtype,
            device=DEVICE,
        )

        # 1.3 Transcribe
        print("   📝 Transcribing...")
        result = pipe(
            target_file,
            chunk_length_s=30,      # Auto Chunking
            batch_size=8,           # Parallel Processing
            return_timestamps=True,
            generate_kwargs={"language": "english"}
        )
        
        text = result["text"].strip()
        
        # 1.4 Cleanup Model Immediately (สำคัญ!)
        del pipe
        cleanup_gpu() 
        
        return text

    except Exception as e:
        print(f"❌ Error STT: {e}")
        return ""
    finally:
        if os.path.exists(temp_wav):
            try: os.remove(temp_wav)
            except: pass

In [5]:
# ---------------------------------------------------------
# 2. TRANSLATION (Typhoon)
# ---------------------------------------------------------
def text_translation(long_text):
    print("------------------------------------------------")
    print("2️⃣ Step 2: Translation (Typhoon 4B)")
    
    if not long_text: return ""

    model_id = "scb10x/typhoon-translate-4b"
    
    try:
        # 2.1 Load Model (Load ใหม่เฉพาะตอนจะใช้)
        print("   🔄 Loading Translation Model...")
        dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
        
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id, 
            torch_dtype=dtype, 
            device_map="auto", # ให้ Library จัดการเอง
        )

        # ฟังก์ชันย่อยสำหรับแปล
        def _translate_chunk(chunk):
            if not chunk.strip(): return ""
            messages = [
                {"role": "system", "content": "Translate the following text into Thai."},
                {"role": "user", "content": chunk},
            ]
            input_ids = tokenizer.apply_chat_template(
                messages, add_generation_prompt=True, return_tensors="pt"
            ).to(model.device)

            with torch.no_grad():
                outputs = model.generate(
                    input_ids, max_new_tokens=512, do_sample=False, pad_token_id=tokenizer.eos_token_id
                )
            return tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

        # 2.2 Split & Translate (Auto Chunking Logic)
        paragraphs = long_text.split('\n')
        full_translation = []
        current_chunk = ""
        MAX_TOKENS = 1200 # ลดลงนิดหน่อยเพื่อความปลอดภัย

        print(f"   📄 Translating {len(paragraphs)} paragraphs...")
        
        for i, para in enumerate(paragraphs):
            if not para.strip():
                if current_chunk:
                    full_translation.append(_translate_chunk(current_chunk))
                    current_chunk = ""
                full_translation.append("")
                continue

            # Check length rough estimate (1 char approx 0.3-0.5 tokens, but len is safer)
            if len(current_chunk) + len(para) > 3000: # ประมาณ 1000 tokens
                 full_translation.append(_translate_chunk(current_chunk))
                 current_chunk = para
            else:
                 current_chunk += "\n" + para if current_chunk else para

        if current_chunk:
            full_translation.append(_translate_chunk(current_chunk))

        # 2.3 Cleanup Model (สำคัญมาก!)
        del model
        del tokenizer
        cleanup_gpu()

        return "\n".join(full_translation)

    except Exception as e:
        print(f"❌ Error Translation: {e}")
        cleanup_gpu()
        return ""

In [6]:
# ---------------------------------------------------------
# 3. TEXT TO SPEECH (Edge TTS)
# ---------------------------------------------------------
def text_to_speech_TH(text, output_path):
    print("------------------------------------------------")
    print("3️⃣ Step 3: Text to Speech (Edge TTS)")
    
    if not text: return

    VOICE = "th-TH-PremwadeeNeural"
    
    async def _gen_chunks(chunks):
        files = []
        for i, chunk in enumerate(chunks):
            if not chunk.strip(): continue
            fname = f"temp_tts_{i}.mp3"
            try:
                # rate="-10%" พูดช้าลงเล็กน้อย
                comm = edge_tts.Communicate(chunk, VOICE, rate="-10%")
                await comm.save(fname)
                files.append(fname)
                print(f"     Generating audio chunk {i+1}...")
            except Exception as e:
                print(f"     ⚠️ Failed chunk {i}: {e}")
        return files

    temp_files = []
    try:
        # หั่นข้อความก่อนส่งไป TTS
        chunks = []
        curr = ""
        for line in text.split('\n'):
            if len(curr) + len(line) < 1000:
                curr += line + "\n"
            else:
                chunks.append(curr)
                curr = line + "\n"
        if curr: chunks.append(curr)

        # Run Async
        loop = asyncio.get_event_loop()
        if loop.is_running():
            temp_files = loop.run_until_complete(_gen_chunks(chunks))
        else:
            temp_files = asyncio.run(_gen_chunks(chunks))

        # Merge Audio
        if temp_files:
            print("   🔗 Merging audio clips...")
            clips = [AudioFileClip(f) for f in temp_files]
            final = concatenate_audioclips(clips)
            final.write_audiofile(output_path, fps=24000, verbose=False, logger=None)
            final.close()
            for c in clips: c.close()
            print(f"   ✅ TTS Saved: {output_path}")
        else:
            print("   ⚠️ No audio generated.")

    except Exception as e:
        print(f"❌ Error TTS: {e}")
    finally:
        # Cleanup temp files
        for f in temp_files:
            if os.path.exists(f): 
                try: os.remove(f)
                except: pass

In [7]:
# ---------------------------------------------------------
# 4. VIDEO DUBBING (FFmpeg)
# ---------------------------------------------------------
def video_sound_editor(video_path, audio_path, output_path):
    print("------------------------------------------------")
    print("4️⃣ Step 4: Dubbing & Sync (FFmpeg)")
    
    # 4.1 ตรวจสอบไฟล์ก่อน (ป้องกัน Error ที่คุณเคยเจอ)
    if not os.path.exists(audio_path):
        print("❌ Error: Audio file not found. Skipping dubbing.")
        return

    vid_dur = get_duration_ffmpeg(video_path)
    aud_dur = get_duration_ffmpeg(audio_path)

    if not vid_dur or not aud_dur:
        print("❌ Error: Cannot read duration.")
        return

    print(f"   ⏱️ Video: {vid_dur:.2f}s | Audio: {aud_dur:.2f}s")

    # 4.2 Calculate Speed
    speed_factor = aud_dur / vid_dur
    if speed_factor < 0.5: speed_factor = 0.5
    if speed_factor > 2.0: speed_factor = 2.0 # Limit extreme speed
    
    print(f"   🔧 Adjusting speed: {speed_factor:.2f}x")

    # 4.3 Execute FFmpeg
    cmd = [
        "ffmpeg", "-y",
        "-i", video_path,
        "-i", audio_path,
        "-filter_complex", f"[1:a]atempo={speed_factor}[aout]",
        "-map", "0:v",
        "-map", "[aout]",
        "-c:v", "copy",
        "-c:a", "aac",
        "-shortest",
        output_path
    ]
    
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"   ✅ DONE! Saved to: {output_path}")
    except subprocess.CalledProcessError:
        print("❌ Error: FFmpeg failed.")

In [8]:
# ---------------------------------------------------------
# GUI & PIPELINE
# ---------------------------------------------------------
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

def show_modern_msg(title, message):
    msg_window = ctk.CTkToplevel()
    msg_window.title(title)
    # จัดหน้าต่างให้อยู่กลางจอ
    w, h = 400, 180
    x = (msg_window.winfo_screenwidth() // 2) - (w // 2)
    y = (msg_window.winfo_screenheight() // 2) - (h // 2)
    msg_window.geometry(f"{w}x{h}+{x}+{y}")
    msg_window.attributes('-topmost', True)

    label = ctk.CTkLabel(msg_window, text=message, font=("Leelawadee UI", 14), wraplength=350)
    label.pack(expand=True, padx=20, pady=20)
    ctk.CTkButton(msg_window, text="OK", command=msg_window.destroy, width=100).pack(pady=(0, 20))
    msg_window.grab_set()
    msg_window.wait_window()

def processing_pipline():
    root = ctk.CTk()
    root.withdraw()

    # 1. เลือกไฟล์
    show_modern_msg("Start", "กรุณาเลือกไฟล์วิดีโอต้นฉบับ")
    video_path = filedialog.askopenfilename(filetypes=[("Video", "*.mp4 *.avi *.mkv *.mov *.webm")])
    if not video_path: return

    base_dir = os.path.dirname(video_path)
    audio_temp_path = os.path.join(base_dir, "temp_dubbing_audio.wav")
    
    try:
        # Step 1: STT
        eng_text = speech_to_text_en(video_path)
        if not eng_text:
            raise Exception("ไม่พบเสียงพูดในวิดีโอ หรือ Whisper ถอดเสียงไม่ได้")

        # Step 2: Translate
        th_text = text_translation(eng_text)
        if not th_text:
            raise Exception("การแปลภาษาล้มเหลว")

        # Step 3: TTS
        text_to_speech_TH(th_text, audio_temp_path)
        if not os.path.exists(audio_temp_path):
             raise Exception("สร้างไฟล์เสียงพากย์ไม่สำเร็จ")

        # Step 4: Dubbing
        show_modern_msg("Save", "ประมวลผลเสร็จสิ้น กรุณาเลือกที่บันทึกไฟล์")
        output_path = filedialog.asksaveasfilename(defaultextension=".mp4", filetypes=[("MP4", "*.mp4")])
        
        if output_path:
            video_sound_editor(video_path, audio_temp_path, output_path)
            show_modern_msg("Success", "เสร็จเรียบร้อย! 🎉")

    except Exception as e:
        print(f"❌ PIPELINE ERROR: {e}")
        show_modern_msg("Error", f"เกิดข้อผิดพลาด:\n{str(e)}")

    # Cleanup Final
    if os.path.exists(audio_temp_path):
        try: os.remove(audio_temp_path)
        except: pass
        
    root.destroy()

In [10]:
if __name__ == "__main__":
    processing_pipline()

------------------------------------------------
1️⃣ Step 1: Speech to Text (Whisper Large-V3)
   🎬 Extracting audio from video...
   🎙️ Loading Whisper Model...


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


   📝 Transcribing...
------------------------------------------------
2️⃣ Step 2: Translation (Typhoon 4B)
   🔄 Loading Translation Model...


`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.66s/it]
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


   📄 Translating 1 paragraphs...
------------------------------------------------
3️⃣ Step 3: Text to Speech (Edge TTS)
   ⚠️ No audio generated.
❌ PIPELINE ERROR: สร้างไฟล์เสียงพากย์ไม่สำเร็จ
